In [1]:
#import some stuff
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import random
import os
from tqdm import tqdm
import glob
import time
import xgboost as xgb
from sklearn.model_selection import train_test_split #use this?
from scipy import stats
from sklearn.metrics import log_loss, mean_squared_error, mean_squared_log_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV #use random search?

In [2]:
pd.set_option('display.max_columns', 500)

In [3]:
cities = pd.read_csv('Cities.csv')
conferences = pd.read_csv('Conferences.csv')
MConfTGames = pd.read_csv('MConferenceTourneyGames.csv')
MGameCities = pd.read_csv('MGameCities.csv')
MMasseyO = pd.read_csv('MMasseyOrdinals.csv')
MTCompactResults = pd.read_csv('MNCAATourneyCompactResults.csv')
MTDetailedResults = pd.read_csv('MNCAATourneyDetailedResults.csv')
MTSeedRoundSlots = pd.read_csv('MNCAATourneySeedRoundSlots.csv')
MTSeeds = pd.read_csv('MNCAATourneySeeds.csv')
MTSlots = pd.read_csv('MNCAATourneySlots.csv')
RegCompactResults = pd.read_csv('MRegularSeasonCompactResults.csv')
RegDetailedResults = pd.read_csv('MRegularSeasonDetailedResults.csv')
MSampleStage1 = pd.read_csv('MSampleSubmissionStage1.csv')
MSampleStage2 = pd.read_csv('MSampleSubmissionStage2.csv')
Mszns = pd.read_csv('MSeasons.csv')
MSecTourneyComRes = pd.read_csv('MSecondaryTourneyCompactResults.csv')
MSecTourneyTeams = pd.read_csv('MSecondaryTourneyTeams.csv')
MTeamConfs = pd.read_csv('MTeamConferences.csv')
#MTeamSpellings = pd.read_csv('MTeamSpellings.csv') #can't read for some reason
MTeamCoaches = pd.read_csv('MTeamCoaches.csv')
MTeams = pd.read_csv('MTeams.csv')

In [4]:
def getAllAvgs(df):
    origDF = df.copy()
    df = df.copy()
    df = df.drop(columns=['DayNum','NumOT'])
    df['Games'] = np.zeros(len(df))
    gamesPlayed = df[['Season','WTeamID','Games']].groupby(by=['Season','WTeamID']).count()
    gamesPlayed2 = df[['Season','LTeamID','Games']].groupby(by=['Season','LTeamID']).count()
    gamesPlayed = gamesPlayed.reset_index()
    gamesPlayed2 = gamesPlayed2.reset_index()
    gamesPlayed = gamesPlayed.merge(gamesPlayed2,how='outer',left_on=['Season','WTeamID'],right_on=['Season','LTeamID'])
    gamesPlayed['Games_x'] = gamesPlayed.apply(lambda row: 0 if pd.isnull(row['Games_x']) else row['Games_x'],axis=1)
    gamesPlayed['Games_y'] = gamesPlayed.apply(lambda row: 0 if pd.isnull(row['Games_y']) else row['Games_y'],axis=1)
    gamesPlayed['TotalGames'] = gamesPlayed['Games_x'] + gamesPlayed['Games_y']
    gamesPlayed['WTeamID'] = gamesPlayed.apply(lambda row: row['LTeamID'] if pd.isnull(row['WTeamID']) else row['WTeamID'],axis=1)
    gamesPlayed = gamesPlayed[['Season','WTeamID','TotalGames']]
    df = df.drop(columns=['Games'])
    TotalWs = df.groupby(by=['Season','WTeamID']).sum().reset_index()
    TotalLs = df.groupby(by=['Season','LTeamID']).sum().reset_index()
    allgames = TotalWs.merge(TotalLs,how='outer',left_on=['Season','WTeamID'],right_on=['Season','LTeamID'])
    allgames = allgames.rename(columns={'WTeamID_x':'WTeamID'})
    allgames['WTeamID'] = allgames.apply(lambda row: row['LTeamID_y'] if pd.isnull(row['WTeamID']) else row['WTeamID'],axis=1)
    allgames = allgames.merge(gamesPlayed,how='left',on=['Season','WTeamID'])
    for col in allgames:
        allgames[col] = allgames.apply(lambda row: 0 if pd.isnull(row[col]) else row[col],axis=1)
    for col in allgames:
        if (col[-1] == "x"):
            if col[0] == "W":
                allgames[col[:-2]] = round((allgames[col] + allgames['L' + col[1:-1] + 'y']) / allgames['TotalGames'],3)
            else:
                allgames[col[:-2]] = round((allgames[col] + allgames['W' + col[1:-1] + 'y']) / allgames['TotalGames'],3)
            
    allgames = allgames[['Season','WTeamID','TotalGames','WScore','WFGM','WFGA','WFGM3','WFGA3','WFTM','WFTA',
                         'WOR','WDR','WAst','WTO','WStl','WBlk','WPF','LScore','LFGM','LFGA','LFGM3','LFGA3','LFTM','LFTA',
                        'LOR','LDR','LAst','LTO','LStl','LBlk','LPF']]
            
    for col in allgames:
        if col[0] == 'W':
            allgames = allgames.rename(columns={col:'Team'+col[1:]})
        elif col[0] == 'L':
            allgames = allgames.rename(columns={col:'Opp'+col[1:]})
    
    allgames = allgames.rename(columns={'TeamTeamID':'TeamID'})
    allgames = allgames.merge(MTeams[['TeamID','TeamName']],how='left',on='TeamID')
    
    #if you just want the averages - comment this all out
    #this gets the over expectation stats
    teamsPlayedList = []
    for i in range(len(allgames)):
        row = allgames.iloc[i]
        tempDF = origDF.loc[(origDF['Season'] == row['Season'])]
        tempDFws = tempDF.loc[(tempDF['WTeamID'] == row['TeamID'])].copy()
        tempDFls = tempDF.loc[(tempDF['LTeamID'] == row['TeamID'])].copy()
        teamsPlayed = tempDFws['LTeamID'].tolist() + tempDFls['WTeamID'].tolist()
        teamsPlayedList.append(teamsPlayed)
    allgames['TeamsPlayed'] = teamsPlayedList
    
    allgames['TeamFGM'] = allgames.apply(lambda row: row['TeamFGM'] - row['TeamFGM3'],axis=1)
    allgames['OppFGM'] = allgames.apply(lambda row: row['OppFGM'] - row['OppFGM3'],axis=1)
    allgames['TeamFGA'] = allgames.apply(lambda row: row['TeamFGA'] - row['TeamFGA3'],axis=1)
    allgames['OppFGA'] = allgames.apply(lambda row: row['OppFGA'] - row['OppFGA3'],axis=1)
    allgames = allgames.rename(columns={'TeamFGM':'TeamFGM2','OppFGM':'OppFGM2','TeamFGA':'TeamFGA2','OppFGA':'OppFGA2'})
    
    overallrows = []
    for i in range(len(allgames)):
        thisrow = allgames.iloc[i]
        opponents = allgames.copy()
        opponents = opponents.loc[opponents['Season'] == thisrow['Season']]
        opponents['PlayedTeam'] = opponents.apply(lambda row: row['TeamID'] in thisrow['TeamsPlayed'],axis=1)
        opponents = opponents.loc[opponents['PlayedTeam'] == True]
        #opponents = opponents.drop(columns=['Season','TeamID','TotalGames','TeamName','TeamsPlayed'])
        opponents = opponents[['OppScore','OppFGM2','OppFGA2','OppFGM3','OppFGA3','OppFTM','OppFTA','OppOR','OppDR','OppAst',
                              'OppTO','OppStl','OppBlk','OppPF','TeamScore','TeamFGM2','TeamFGA2','TeamFGM3','TeamFGA3',
                               'TeamFTM','TeamFTA','TeamOR','TeamDR','TeamAst','TeamTO','TeamStl','TeamBlk','TeamPF']]
        oppAvgs = opponents.mean()
        tempList = [thisrow['Season'],thisrow['TeamID'],thisrow['TeamName'],thisrow['TotalGames']]
        for real,exp in zip(thisrow.tolist()[3:31],oppAvgs):
            tempList.append(real - exp)
        overallrows.append(tempList)
    cols = ['Season','TeamID','TeamName','TotalGames','TeamScoreOE','TeamFGM2OE','TeamFGA2OE','TeamFGM3OE','TeamFGA3OE','TeamFTMOE','TeamFTAOE',
            'TeamOROE','TeamDROE','TeamAstOE','TeamTOOE','TeamStlOE','TeamBlkOE','TeamPFOE','OppScoreOE','OppFGM2OE','OppFGA2OE',
            'OppFGM3OE','OppFGA3OE','OppFTMOE','OppFTAOE','OppOROE','OppDROE','OppAstOE','OppTOOE','OppStlOE','OppBlkOE','OppPFOE']
    
    finalDF = pd.DataFrame(overallrows,columns=cols)
    
    return finalDF

In [5]:
%%time
allAvg = getAllAvgs(RegDetailedResults)

Wall time: 3min 47s


In [6]:
just2021 = RegDetailedResults.loc[RegDetailedResults['Season'] == 2021].copy()
allAvg2021 = getAllAvgs(just2021)

In [7]:
allAvg.head(1)

,Season,TeamID,TeamName,TotalGames,TeamScoreOE,TeamFGM2OE,TeamFGA2OE,TeamFGM3OE,TeamFGA3OE,TeamFTMOE,TeamFTAOE,TeamOROE,TeamDROE,TeamAstOE,TeamTOOE,TeamStlOE,TeamBlkOE,TeamPFOE,OppScoreOE,OppFGM2OE,OppFGA2OE,OppFGM3OE,OppFGA3OE,OppFTMOE,OppFTAOE,OppOROE,OppDROE,OppAstOE,OppTOOE,OppStlOE,OppBlkOE,OppPFOE
0,2003.0,1102.0,Air Force,28.0,-14.019053,-7.059789,-18.284368,1.562737,3.101684,-4.587895,-5.469579,-7.191789,-6.319368,-0.962632,-3.920842,-1.104316,-1.422737,-1.200211,-13.981947,-4.168737,-8.413368,-1.415105,-5.287579,-1.397947,-2.519105,-2.282368,-2.614,-4.393211,-2.150158,-1.953421,-1.378526,-1.839895


In [8]:
#MMasseyO['SystemName'].unique() #'HOL','POM','MAS','BPI' are good

In [36]:
#this is used in finishCleaning below
mas = MMasseyO.loc[MMasseyO['SystemName'] == 'MAS']
mas = mas.drop_duplicates(subset=['Season','TeamID'],keep='last')
mas = mas[['Season','TeamID','OrdinalRank']]

def finishCleaning(tourneygames,seasonAvgs,includeSeeds,includeOrdinals):
    tourneygames = tourneygames.copy()
    seasonAvgs = seasonAvgs.copy()
    tourneygames = tourneygames.loc[tourneygames['Season'] > 2002]
    tourneygames = tourneygames.drop(columns=['NumOT','DayNum','WLoc'])
    random.seed(0)
    tourneygames['WinningTeam'] = tourneygames.apply(lambda row: random.randint(0,1),axis=1)
    tourneygames['Team0'] = tourneygames.apply(lambda row: row['WTeamID'] if (row['WinningTeam'] == 0) 
                                               else row['LTeamID'],axis=1)
    tourneygames['Team1'] = tourneygames.apply(lambda row: row['LTeamID'] if (row['WinningTeam'] == 0) 
                                               else row['WTeamID'],axis=1)
    #I could add stuff for point diff but we just predicting winner
    tourneygames = tourneygames[['Season','Team0','Team1','WinningTeam']]
    seasonAvgs = seasonAvgs.drop(columns=['TeamName'])
    tourneygames = tourneygames.merge(seasonAvgs,how='left',left_on=['Season','Team0'],right_on=['Season','TeamID'])
    tourneygames = tourneygames.merge(seasonAvgs,how='left',left_on=['Season','Team1'],right_on=['Season','TeamID'],suffixes=('0','1'))
    tourneygames = tourneygames.drop(columns=['TeamID0','TotalGames0','TeamID1','TotalGames1'])
    y = tourneygames[['WinningTeam']]
    if includeSeeds:
        #this just uses the global MTSeeds dataframe
        tourneygames = tourneygames.merge(MTSeeds,how='left',left_on=['Season','Team0'],right_on=['Season','TeamID'])
        tourneygames = tourneygames.merge(MTSeeds,how='left',left_on=['Season','Team1'],right_on=['Season','TeamID'],suffixes=('0','1'))
        tourneygames['Seed0'] = tourneygames.apply(lambda row: float(row['Seed0'][1:3]),axis=1)
        tourneygames['Seed1'] = tourneygames.apply(lambda row: float(row['Seed1'][1:3]),axis=1)
        if includeOrdinals:
            tourneygames = tourneygames.merge(mas,how='left',left_on=['Season','Team0'],right_on=['Season','TeamID'])
            tourneygames = tourneygames.merge(mas,how='left',left_on=['Season','Team1'],right_on=['Season','TeamID'],suffixes=('00','01'))
            X = tourneygames.drop(columns=['WinningTeam','TeamID0','TeamID1','TeamID00','TeamID01'])
        else:
            X = tourneygames.drop(columns=['WinningTeam','TeamID0','TeamID1'])
    else:
        if includeOrdinals:
            tourneygames = tourneygames.merge(mas,how='left',left_on=['Season','Team0'],right_on=['Season','TeamID'])
            tourneygames = tourneygames.merge(mas,how='left',left_on=['Season','Team1'],right_on=['Season','TeamID'],suffixes=('00','01'))
            X = tourneygames.drop(columns=['WinningTeam','TeamID00','TeamID01'])
        else:
            X = tourneygames.drop(columns=['WinningTeam'])
    X = X.drop(columns=['Season','Team0','Team1']) #use this when not using Will's sim
    return X,y

In [38]:
X,y = finishCleaning(MTCompactResults,allAvg,True,False)
X

,TeamScoreOE0,TeamFGM2OE0,TeamFGA2OE0,TeamFGM3OE0,TeamFGA3OE0,TeamFTMOE0,TeamFTAOE0,TeamOROE0,TeamDROE0,TeamAstOE0,TeamTOOE0,TeamStlOE0,TeamBlkOE0,TeamPFOE0,OppScoreOE0,OppFGM2OE0,OppFGA2OE0,OppFGM3OE0,OppFGA3OE0,OppFTMOE0,OppFTAOE0,OppOROE0,OppDROE0,OppAstOE0,OppTOOE0,OppStlOE0,OppBlkOE0,OppPFOE0,TeamScoreOE1,TeamFGM2OE1,TeamFGA2OE1,TeamFGM3OE1,TeamFGA3OE1,TeamFTMOE1,TeamFTAOE1,TeamOROE1,TeamDROE1,TeamAstOE1,TeamTOOE1,TeamStlOE1,TeamBlkOE1,TeamPFOE1,OppScoreOE1,OppFGM2OE1,OppFGA2OE1,OppFGM3OE1,OppFGA3OE1,OppFTMOE1,OppFTAOE1,OppOROE1,OppDROE1,OppAstOE1,OppTOOE1,OppStlOE1,OppBlkOE1,OppPFOE1,Seed0,Seed1
0,1.391278,-0.241833,-2.438556,-0.182167,0.723111,2.420500,6.079389,0.720278,0.230833,0.204278,-0.435111,-1.005389,-0.926333,-0.468278,2.313278,0.004333,-2.056278,1.419389,5.015111,-1.952500,-1.837333,-0.172667,-0.267222,0.876722,-1.378944,0.408389,-0.146556,2.239556,1.886556,-0.350944,0.017278,0.295278,0.396778,1.702556,0.566778,0.494167,0.679500,-0.177778,1.079000,-0.554722,-0.564389,-0.041167,8.182167,1.379278,1.180722,1.660222,3.681889,0.443056,1.452222,1.537222,-0.644500,2.334389,-2.723389,1.341500,0.234722,-0.255000,16.0,16.0
1,-2.311722,0.733111,2.252389,-1.011056,-2.662667,-0.743333,-0.141056,1.221444,1.825222,0.328056,-0.503944,-0.496833,-0.392389,-2.381056,-4.518722,-1.749556,-3.180778,0.536056,2.435778,-2.626833,-3.309056,-1.652667,-0.954944,-0.276833,-1.712889,0.164722,0.525944,-0.608333,15.017333,4.551944,5.808167,0.866278,2.050667,3.314889,4.517278,3.419056,4.706722,4.348278,-0.182000,1.729778,1.038444,-1.751556,-3.243444,0.233444,4.387167,-0.191500,1.746611,-3.136833,-3.055222,0.731333,-0.900611,0.704167,2.066556,-0.598278,-0.892111,2.827611,16.0,1.0
2,6.156278,4.449278,4.849556,-2.048611,-5.389111,3.403611,5.835389,1.955333,0.149667,1.990500,-0.990556,-1.067722,1.189556,-0.412111,-3.485333,0.056778,-0.007778,-0.853167,-1.980611,-1.039389,-1.023222,-1.008889,-3.453111,0.049333,1.059611,-0.598222,0.662333,3.787111,6.408952,1.363619,1.655048,0.932429,1.781143,0.885714,2.316476,1.958238,2.482095,3.250238,-0.795714,0.504619,1.777810,-0.163810,-3.329476,-1.032333,1.176762,-0.246286,-0.051476,-0.525857,0.327429,0.172190,-0.355571,-0.244000,0.508238,0.293429,-0.471952,0.905810,10.0,7.0
3,10.349952,2.452762,-0.753381,1.686476,2.742000,0.386143,0.759095,-0.026286,0.668524,3.318619,-0.672190,1.633524,1.260619,-1.876857,-4.850571,0.737714,3.127476,-1.166524,-2.459429,-2.825667,-3.813429,-0.125429,-1.868667,-1.429238,2.588286,-0.121381,-0.279571,0.775667,9.221600,1.463700,-1.770600,0.623650,0.202350,4.424350,4.027200,-0.057950,0.622650,1.932200,3.996200,0.126150,0.962250,1.131650,3.676500,3.290800,6.877700,-1.078200,-3.267500,0.330350,0.297600,1.119600,-3.736700,-1.313100,1.613200,1.812800,-0.124200,2.089400,6.0,11.0
4,2.444526,-1.427263,-6.444947,1.114316,3.025158,1.956316,0.627684,-2.726632,-1.139000,0.683105,-1.045421,0.641895,-0.361842,-0.422421,-2.425000,-0.626368,-1.776947,-0.856474,-2.146211,1.396263,0.788053,-1.621211,-1.252158,-1.310895,0.218316,-0.128263,-0.816684,0.666684,2.390167,1.355778,1.277056,0.175667,-0.928278,-0.848444,-1.065667,-0.958778,0.893222,1.959611,-1.070000,0.085944,-0.311278,-2.437667,-4.021056,-1.608389,-3.551389,0.692333,3.300667,-2.882222,-3.788444,-1.282722,-0.494056,1.570222,0.317056,-0.885000,-0.841444,0.019389,9.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1110,8.784640,-1.766360,-5.428360,4.028680,8.042800,0.231120,-0.009040,1.027400,-1.492360,1.611840,-1.165840,2.899360,1.345120,0.472720,-4.328040,-3.669920,-6.239960,1.005240,1.831040,-0.003960,-0.146720,0.034200,-0.835080,-1.493800,4.244600,-0.607000,-0.509760,-0.416520,4.921731,3.114462,3.384308,-1.707615,-6.009038,3.815615,4.317154,1.233577,2.419269,1.249346,-0.834885,-0.367308,1.522308,-1.892308,-8.804346,-2.691000,-0.030615,-0.126462,-0.6513

In [27]:
def makeModel(useSeeds,useOrds,justSeeds,modelname):
    X,y = finishCleaning(MTCompactResults,allAvg,True,useOrds)
    print("Overall Mean:", y['WinningTeam'].mean())
    #get train/test split ready

    if justSeeds:
        X = X[['Seed0','Seed1']].copy()
    
    Xtrain, Xtest, ytrain, ytest = train_test_split(X,y,test_size=0.25,random_state=24) #can add shuffle=False to not shuffle
    
    seeds0 = Xtest['Seed0'].tolist()
    seeds1 = Xtest['Seed1'].tolist()
    if not useSeeds:
        Xtrain = Xtrain.drop(columns=['Seed0','Seed1'])
        Xtest = Xtest.drop(columns=['Seed0','Seed1'])

    #train the xgboost model
    dtrain = xgb.DMatrix(Xtrain,label=ytrain['WinningTeam'])
    # XGBoost model
    nrounds = 560 #use gridSearchCV
    params = {
        'objective': 'binary:logistic',
        'booster' : 'gblinear', #gblinear/gbtree?
        'eval_metric' : ['logloss'],
        'eta' : 0.05,
        'gamma' : 3,
        'subsample' : 0.8,
        'colsample_bytree' : 0.8,
        'max_depth': 6,
        'min_child_weight' : 6,
        'base_score' : ytrain['WinningTeam'].mean()
    }
    
    myModel = xgb.train(params, dtrain, num_boost_round=nrounds, verbose_eval=2)
    # save for debug
    myModel.save_model(modelname)

    #test the model
    dtest = xgb.DMatrix(Xtest,label=ytest['WinningTeam'])
    xgb_predictions = myModel.predict(dtest).tolist()
    print("XGBoost mean vs actual mean:")
    XtestCopy = Xtest.copy()
    XtestCopy['x1Wins'] = xgb_predictions
    print(XtestCopy['x1Wins'].mean())
    print(ytest['WinningTeam'].mean())

    #now let's get the errors
    #this is for kaggle
    LLerror = log_loss(ytest,xgb_predictions)
    print("Log loss error is:",LLerror)

    #this is for the minneanalytics
    correct = 0
    incorrect = 0
    upsetPoints = 0
    ytestList = ytest['WinningTeam'].tolist()
    for x1,y1,seed0,seed1 in zip(ytestList,xgb_predictions,seeds0,seeds1):
        if abs(x1-y1) <= 0.5:
            if (x1 == 0) and (seed0 > seed1):
                upsetPoints += (seed0 - seed1)
            elif (x1 == 1) and (seed1 > seed0):
                upsetPoints += (seed1 - seed0)
            correct +=1
        else:
            incorrect +=1
    print('correct percent is:',round(100*correct/(correct+incorrect),2),'%')
    print('Upset Points + correct percent is:',round(100*(correct+upsetPoints)/(correct+incorrect),2),'%')
    print('Upset Points:',upsetPoints)
    return myModel

In [35]:
#this model has just the over expectation data
justOEModel = makeModel(False,False,False,'justOEModel.json')

Overall Mean: 0.5013452914798207
XGBoost mean vs actual mean:
0.5099916445712248
0.5089605734767025
Log loss error is: 0.5377925581567459
correct percent is: 70.25 %
Upset Points + correct percent is: 92.47 %
Upset Points: 62.0


In [29]:
#this model has just seeds (no over expectation data)
justSeedsModel = makeModel(True,False,True,'justSeedsModel.json')

Overall Mean: 0.5013452914798207
XGBoost mean vs actual mean:
0.508780320805888
0.5089605734767025
Log loss error is: 0.5265811850360271
correct percent is: 74.55 %
Upset Points + correct percent is: 76.34 %
Upset Points: 5.0


In [30]:
#this model has seeds and ordinals
SeedsAndOrdsModel = makeModel(True,True,False,'SeedsAndOrdsModel.json')

Overall Mean: 0.5013452914798207
XGBoost mean vs actual mean:
0.5086272403924963
0.5089605734767025
Log loss error is: 0.52692853356029
correct percent is: 69.89 %
Upset Points + correct percent is: 75.63 %
Upset Points: 16.0


In [31]:
#this model has seeds and no ordinals
SeedsAndNoOrdsModel = makeModel(True,False,False,'SeedsAndNoOrdsModel.json')

Overall Mean: 0.5013452914798207
XGBoost mean vs actual mean:
0.5081322728305735
0.5089605734767025
Log loss error is: 0.5267183366130488
correct percent is: 70.97 %
Upset Points + correct percent is: 77.06 %
Upset Points: 17.0


In [32]:
#this model has ordinals but no seeds
OrdsAndNoSeedsModel = makeModel(False,True,False,'OrdsAndNoSeedsModel.json')

Overall Mean: 0.5013452914798207
XGBoost mean vs actual mean:
0.5099343214881703
0.5089605734767025
Log loss error is: 0.5325751331129197
correct percent is: 69.53 %
Upset Points + correct percent is: 84.95 %
Upset Points: 43.0


In [33]:
def TourneySim(Model, TeamsStats, TourneySeeds, TourneySlots, Teams, Ordinals, Year):
    
    SeasonSeeds = TourneySeeds.loc[MTSeeds['Season'] == Year][['Seed', 'TeamID']] # TourneySeeds should = MTSeeds
    
    SeasonSlots = TourneySlots.loc[MTSlots['Season'] == Year][['Slot', 'StrongSeed', 'WeakSeed']] # TourneySlots should = MTSlots
                                    
    Winners = {}
    
    
    
    def MergeData(Team0, Team1):
        Team0df = TeamsStats.loc[(TeamsStats['Season'] == Year) & (TeamsStats['TeamID'] == Team0)]
        Team1df = TeamsStats.loc[(TeamsStats['Season'] == Year) & (TeamsStats['TeamID'] == Team1)]
        Team0df = Team0df.drop(columns=['TeamName', 'TotalGames'])
        Team1df = Team1df.drop(columns=['TeamName', 'TotalGames'])
        MergedData = Team0df.merge(Team1df, how='left', left_on=['Season'],right_on=['Season'], suffixes=('0','1')) #MergeData(Team0,Team1) # merged the data of both teams as formatted as the model was trained
        if Ordinals is not None:
            MergedData = MergedData.merge(Ordinals,how='left',left_on=['Season','TeamID0'],right_on=['Season','TeamID'])
            MergedData = MergedData.merge(Ordinals,how='left',left_on=['Season','TeamID1'],right_on=['Season','TeamID'],suffixes=('00','01'))
            MergedData = MergedData.drop(columns=['TeamID00', 'TeamID01'])
        MergedData = MergedData.drop(columns=['Season', 'TeamID0', 'TeamID1'])
        MergedData = xgb.DMatrix(MergedData)
            
        
        return MergedData
    
    
    def GameSim(Game, GameList): # sims a game, but GameSim('R6CH', []) sims the whole tourney
        
        Slot0 = SeasonSlots.loc[SeasonSlots['Slot'] == Game].iloc[0]['StrongSeed'] # Strong Slot
        Slot1 = SeasonSlots.loc[SeasonSlots['Slot'] == Game].iloc[0]['WeakSeed'] # Weak Slot
        
        if Slot0 in SeasonSeeds[['Seed']].values: # Gets TeamID of StrongSlot
            Team0 = SeasonSeeds[SeasonSeeds['Seed'] == Slot0].iloc[0]['TeamID'] #if Slot = a Seed, get TeamID from SeasonSeeds
            Seed0 = int(Slot0[1:3]) # if Slot = a Seed, than remove 1st (and 4th) char to get seed
        else:
            Team0 = GameSim(Slot0, [Game] + GameList) #if Slot != a Seed, it's a game -> sim this game
            Seed0 = int(SeasonSeeds.loc[SeasonSeeds['TeamID'] == Team0].iloc[0]['Seed'][1:3]) # get seed based on TeamID
        
        if Slot1 in SeasonSeeds[['Seed']].values:
            Team1 = SeasonSeeds[SeasonSeeds['Seed'] == Slot1].iloc[0]['TeamID']
            Seed1= int(Slot1[1:3])
        else:
            Team1 = GameSim(Slot1, [Game] + GameList)
            Seed1 = int(SeasonSeeds.loc[SeasonSeeds['TeamID'] == Team1].iloc[0]['Seed'][1:3])
        
        #print(Team1)
        
        MergedData = MergeData(Team0, Team1)
        
        p = Model.predict(MergedData)[0] # p = chance team 1 will win (I think)
        #p = 0
        
        if len(GameList) == 6:
            RoundScore = 0 # play in rounds have 0 points I believe
        else:
            RoundScore = 2 ** (5 - len(GameList)) # 6 rounds -> round 1: 1 point, round 2: 2 points, round 3: 4 points ...
        
        E0 = (1 - p) * (RoundScore + max(0, Seed0 - Seed1) + SimFuture(Team0, Game, GameList)) # Expected Value Team 0 wins
        
        E1 = p * (RoundScore + max(0, Seed1 - Seed0) + SimFuture(Team1, Game, GameList)) # Expect Value Team 1 Wins
        
        if E1 > E0: # return team with highest EV, if somehow a tie, it goes to "strong" seed
            TeamName = Teams.loc[Teams['TeamID'] == Team1].iloc[0]['TeamName']
            Winners[Game] = TeamName
            return Team1
        else:
            TeamName = Teams.loc[Teams['TeamID'] == Team0].iloc[0]['TeamName']
            Winners[Game] = TeamName
            return Team0
        
        
        
    def SimFuture(Team, PrevGame, GameList): # sims future games by comparing chances to Strongest Seed
        if len(GameList) == 0:
            return 0
        
        # Team is treated as Team1
        
        CurrGame = GameList[0]
        GamesLeft = GameList[1:]
        SlotS = SeasonSlots.loc[SeasonSlots['Slot'] == CurrGame].iloc[0]['StrongSeed']
        SlotW = SeasonSlots.loc[SeasonSlots['Slot'] == CurrGame].iloc[0]['WeakSeed']
        
        if SlotS == PrevGame: # Set Slot0 to the opposite slot of PrevGame
            Slot0 = SlotW
        else:
            Slot0 = SlotS
        
        while Slot0 not in SeasonSeeds[['Seed']].values: # Gets "Strongest" Seed
            Slot0 =  SeasonSlots.loc[SeasonSlots['Slot'] == Slot0].iloc[0]['StrongSeed']
        
        Team0 = SeasonSeeds[SeasonSeeds['Seed'] == Slot0].iloc[0]['TeamID']
        Seed0 = int(SeasonSeeds.loc[SeasonSeeds['TeamID'] == Team0].iloc[0]['Seed'][1:3])
        
        Seed1 = int(SeasonSeeds.loc[SeasonSeeds['TeamID'] == Team].iloc[0]['Seed'][1:3])
                
        MergedData = MergeData(Team0, Team)
        
        p = Model.predict(MergedData)[0]
        
        #p = 0.4
        
        if len(GamesLeft) == 6:
            RoundScore = 0
        else:
            RoundScore = 2 ** (5 - len(GamesLeft))
        
        return p * (RoundScore + max(0, Seed1 - Seed0) + SimFuture(Team, CurrGame, GamesLeft)) # returns EV if Team wins
    
    GameSim('R6CH', [])
    
    for [Key] in SeasonSlots[['Slot']].values:
        print(f'{Key}: {Winners[Key]}')
        
        

In [34]:
%%time
#this is for our predictions
TourneySim(OrdsAndNoSeedsModel, allAvg2021, MTSeeds, MTSlots, MTeams, mas, 2021)

R1W1: Michigan
R1W2: Alabama
R1W3: Abilene Chr
R1W4: UNC Greensboro
R1W5: Colorado
R1W6: UCLA
R1W7: Connecticut
R1W8: St Bonaventure
R1X1: Gonzaga
R1X2: Iowa
R1X3: E Washington
R1X4: Ohio
R1X5: UC Santa Barbara
R1X6: Drake
R1X7: VCU
R1X8: Oklahoma
R1Y1: Drexel
R1Y2: Houston
R1Y3: Morehead St
R1Y4: Liberty
R1Y5: Tennessee
R1Y6: San Diego St
R1Y7: Rutgers
R1Y8: Loyola-Chicago
R1Z1: Baylor
R1Z2: Oral Roberts
R1Z3: Colgate
R1Z4: North Texas
R1Z5: Winthrop
R1Z6: Texas Tech
R1Z7: Virginia Tech
R1Z8: Wisconsin
R2W1: St Bonaventure
R2W2: Connecticut
R2W3: Abilene Chr
R2W4: Colorado
R2X1: Gonzaga
R2X2: Iowa
R2X3: Drake
R2X4: UC Santa Barbara
R2Y1: Loyola-Chicago
R2Y2: Houston
R2Y3: San Diego St
R2Y4: Tennessee
R2Z1: Baylor
R2Z2: Virginia Tech
R2Z3: Texas Tech
R2Z4: Winthrop
R3W1: St Bonaventure
R3W2: Connecticut
R3X1: Gonzaga
R3X2: Drake
R3Y1: Loyola-Chicago
R3Y2: Houston
R3Z1: Baylor
R3Z2: Texas Tech
R4W1: Connecticut
R4X1: Gonzaga
R4Y1: Houston
R4Z1: Baylor
R5WX: Gonzaga
R5YZ: Houston
R6CH: H